In [1]:
import torch
import torch.nn as nn
import torch_geometric.nn as nng
import random


import argparse, yaml, os, json, glob
import torch
import train, metrics
from dataset import Dataset

import numpy as np

/home/jessica/anaconda3/envs/airfrans_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Redefining GUNet.py to use aSGCN convolutions**

Original code: https://github.com/Extrality/AirfRANS/blob/main/models/GUNet.py


In [2]:
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import time, json

import torch
import torch.nn as nn
import torch_geometric.nn as nng
from torch_geometric.loader import DataLoader

import metrics

from tqdm import tqdm

def get_nb_trainable_params(model):
   '''
   Return the number of trainable parameters
   '''
   model_parameters = filter(lambda p: p.requires_grad, model.parameters())
   return sum([np.prod(p.size()) for p in model_parameters])

def train(device, model, train_loader, optimizer, scheduler, criterion = 'MSE', reg = 1):
    model.train()
    avg_loss_per_var = torch.zeros(4, device = device)
    avg_loss = 0
    avg_loss_surf_var = torch.zeros(4, device = device)
    avg_loss_vol_var = torch.zeros(4, device = device)
    avg_loss_surf = 0
    avg_loss_vol = 0
    iter = 0
    
    for data in train_loader:
        data_clone = data.clone()
        data_clone = data_clone.to(device)          
        optimizer.zero_grad()  
        out = model(data_clone)
        targets = data_clone.y

        if criterion == 'MSE' or criterion == 'MSE_weighted':
            criterion = nn.MSELoss(reduction = 'none')
        elif criterion == 'MAE':
            criterion = nn.L1Loss(reduction = 'none')
        loss_per_var = criterion(out, targets).mean(dim = 0)
        total_loss = loss_per_var.mean()
        loss_surf_var = criterion(out[data_clone.surf, :], targets[data_clone.surf, :]).mean(dim = 0)
        loss_vol_var = criterion(out[~data_clone.surf, :], targets[~data_clone.surf, :]).mean(dim = 0)
        loss_surf = loss_surf_var.mean()
        loss_vol = loss_vol_var.mean() 
        if criterion == 'MSE_weighted':            
            (loss_vol + reg*loss_surf).backward()           
        else:
            total_loss.backward()
        
        optimizer.step()
        scheduler.step()
        avg_loss_per_var += loss_per_var
        avg_loss += total_loss
        avg_loss_surf_var += loss_surf_var
        avg_loss_vol_var += loss_vol_var
        avg_loss_surf += loss_surf
        avg_loss_vol += loss_vol 
        iter += 1

    return avg_loss.cpu().data.numpy()/iter, avg_loss_per_var.cpu().data.numpy()/iter, avg_loss_surf_var.cpu().data.numpy()/iter, avg_loss_vol_var.cpu().data.numpy()/iter, \
            avg_loss_surf.cpu().data.numpy()/iter, avg_loss_vol.cpu().data.numpy()/iter

@torch.no_grad()
def test(device, model, test_loader, criterion = 'MSE'):
    model.eval()
    avg_loss_per_var = np.zeros(4)
    avg_loss = 0
    avg_loss_surf_var = np.zeros(4)
    avg_loss_vol_var = np.zeros(4)
    avg_loss_surf = 0
    avg_loss_vol = 0
    iter = 0

    for data in test_loader:        
        data_clone = data.clone()
        data_clone = data_clone.to(device)
        out = model(data_clone)       

        targets = data_clone.y
        if criterion == 'MSE' or 'MSE_weighted':
            criterion = nn.MSELoss(reduction = 'none')
        elif criterion == 'MAE':
            criterion = nn.L1Loss(reduction = 'none')

        loss_per_var = criterion(out, targets).mean(dim = 0)
        loss = loss_per_var.mean()
        loss_surf_var = criterion(out[data_clone.surf, :], targets[data_clone.surf, :]).mean(dim = 0)
        loss_vol_var = criterion(out[~data_clone.surf, :], targets[~data_clone.surf, :]).mean(dim = 0)
        loss_surf = loss_surf_var.mean()
        loss_vol = loss_vol_var.mean()  

        avg_loss_per_var += loss_per_var.cpu().numpy()
        avg_loss += loss.cpu().numpy()
        avg_loss_surf_var += loss_surf_var.cpu().numpy()
        avg_loss_vol_var += loss_vol_var.cpu().numpy()
        avg_loss_surf += loss_surf.cpu().numpy()
        avg_loss_vol += loss_vol.cpu().numpy()  
        iter += 1
    
    return avg_loss/iter, avg_loss_per_var/iter, avg_loss_surf_var/iter, avg_loss_vol_var/iter, avg_loss_surf/iter, avg_loss_vol/iter

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

def main(device, train_dataset, val_dataset, Net, hparams, path, criterion = 'MSE', reg = 1, val_iter = 10, name_mod = 'GraphSAGE', val_sample = False, use_saf = False,use_dsdf = False):
    '''
        Args:
        device (str): device on which you want to do the computation.
        train_dataset (list): list of the data in the training set.
        val_dataset (list): list of the data in the validation set.
        Net (class): network to train.
        hparams (dict): hyper parameters of the network.
        path (str): where to save the trained model and the figures.
        criterion (str, optional): chose between 'MSE', 'MAE', and 'MSE_weigthed'. The latter is the volumetric MSE plus the surface MSE computed independently. Default: 'MSE'.
        ref (float, optional): weigth for the surface loss when criterion is 'MSE_weighted'. Default: 1.
        val_iter (int, optional): number of epochs between each validation step. Default: 10.
    '''

    model = Net.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr = hparams['lr'])
    lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer,
            max_lr = hparams['lr'],
            total_steps = (len(train_dataset) // hparams['batch_size'] + 1) * hparams['nb_epochs'],
        )
    val_loader = DataLoader(val_dataset, batch_size = 1)
    start = time.time()

    train_loss_surf_list = []
    train_loss_vol_list = []
    loss_surf_var_list = []
    loss_vol_var_list = []
    val_surf_list = []
    val_vol_list = []
    val_surf_var_list = []
    val_vol_var_list = []

    pbar_train = tqdm(range(hparams['nb_epochs']), position=0)
    for epoch in pbar_train:        
        train_dataset_sampled = []
        for data in train_dataset:
            data_sampled = data.clone()
            idx = random.sample(range(data_sampled.x.size(0)), hparams['subsampling'])
            idx = torch.tensor(idx)

            data_sampled.pos = data_sampled.pos[idx]
            data_sampled.x = data_sampled.x[idx]
            data_sampled.y = data_sampled.y[idx]
            data_sampled.surf = data_sampled.surf[idx]
            if (use_saf):
                data_sampled.saf = data_sampled.saf[idx]
            if (use_dsdf):
                # for i in range(data_sampled.dsdf.size(0)):
                # data_sampled.dsdf[i] = data_sampled.dsdf[i][idx]
                data_sampled.dsdf = data_sampled.dsdf[:,idx]
            if name_mod != 'PointNet' and name_mod != 'MLP':
                data_sampled.edge_index = nng.radius_graph(x = data_sampled.pos.to(device), r = hparams['r'], loop = True, max_num_neighbors = int(hparams['max_neighbors'])).cpu()

                # if name_mod == 'GNO' or name_mod == 'MGNO':
                #     x, edge_index = data_sampled.x, data_sampled.edge_index
                #     x_i, x_j = x[edge_index[0], 0:2], x[edge_index[1], 0:2]
                #     v_i, v_j = x[edge_index[0], 2:4], x[edge_index[1], 2:4]
                #     p_i, p_j = x[edge_index[0], 4:5], x[edge_index[1], 4:5]
                #     v_inf = torch.linalg.norm(v_i, dim = 1, keepdim = True)
                #     sdf_i, sdf_j = x[edge_index[0], 5:6], x[edge_index[1], 5:6]
                #     normal_i, normal_j = x[edge_index[0], 6:8], x[edge_index[1], 6:8]

                #     data_sampled.edge_attr = torch.cat([x_i - x_j, v_i - v_j, p_i - p_j, sdf_i, sdf_j, v_inf, normal_i, normal_j], dim = 1)
            
            train_dataset_sampled.append(data_sampled)
        train_loader = DataLoader(train_dataset_sampled, batch_size = hparams['batch_size'], shuffle = True)
        # for data in train_dataset_sampled:
        #     assert(data.x.size(0)==data.saf.size(0))
        del(train_dataset_sampled)

        _, _, loss_surf_var, loss_vol_var, loss_surf, loss_vol = train(device, model, train_loader, optimizer, lr_scheduler, criterion, reg = reg)
        del(train_loader)
        train_loss = loss_surf + loss_vol
        train_loss_surf_list.append(loss_surf)
        train_loss_vol_list.append(loss_vol)
        loss_surf_var_list.append(loss_surf_var)
        loss_vol_var_list.append(loss_vol_var)
  
        if val_iter is not None:
            if epoch%val_iter == val_iter - 1 or epoch == 0:
                if val_sample:
                    val_surf_vars, val_vol_vars, val_surfs, val_vols = [], [], [], []
                    for i in range(20):
                        val_dataset_sampled = []
                        for data in val_dataset:
                            data_sampled = data.clone()
                            idx = random.sample(range(data_sampled.x.size(0)), hparams['subsampling'])
                            idx = torch.tensor(idx)

                            data_sampled.pos = data_sampled.pos[idx]
                            data_sampled.x = data_sampled.x[idx]
                            data_sampled.y = data_sampled.y[idx]
                            data_sampled.surf = data_sampled.surf[idx]
                            if (use_saf):
                                data_sampled.saf = data_sampled.saf[idx]
                            if (use_dsdf):
                                # for i in range(data_sampled.dsdf.size(0)):
                                data_sampled.dsdf = data_sampled.dsdf[:,idx]
                            if name_mod != 'PointNet' and name_mod != 'MLP':
                                data_sampled.edge_index = nng.radius_graph(x = data_sampled.pos.to(device), r = hparams['r'], loop = True, max_num_neighbors = int(hparams['max_neighbors'])).cpu()

                                # if name_mod == 'GNO' or name_mod == 'MGNO':
                                #     x, edge_index = data_sampled.x, data_sampled.edge_index
                                #     x_i, x_j = x[edge_index[0], 0:2], x[edge_index[1], 0:2]
                                #     v_i, v_j = x[edge_index[0], 2:4], x[edge_index[1], 2:4]
                                #     p_i, p_j = x[edge_index[0], 4:5], x[edge_index[1], 4:5]
                                #     v_inf = torch.linalg.norm(v_i, dim = 1, keepdim = True)
                                #     sdf_i, sdf_j = x[edge_index[0], 5:6], x[edge_index[1], 5:6]
                                #     normal_i, normal_j = x[edge_index[0], 6:8], x[edge_index[1], 6:8]

                                #     data_sampled.edge_attr = torch.cat([x_i - x_j, v_i - v_j, p_i - p_j, sdf_i, sdf_j, v_inf, normal_i, normal_j], dim = 1)
                            
                            val_dataset_sampled.append(data_sampled)
                        val_loader = DataLoader(val_dataset_sampled, batch_size = 1, shuffle = True)
                        del(val_dataset_sampled)

                        _, _, val_surf_var, val_vol_var, val_surf, val_vol = test(device, model, val_loader, criterion)
                        del(val_loader)
                        val_surf_vars.append(val_surf_var)
                        val_vol_vars.append(val_vol_var)
                        val_surfs.append(val_surf)
                        val_vols.append(val_vol)
                    val_surf_var = np.array(val_surf_vars).mean(axis = 0)
                    val_vol_var = np.array(val_vol_vars).mean(axis = 0)
                    val_surf = np.array(val_surfs).mean(axis = 0)
                    val_vol = np.array(val_vols).mean(axis = 0)
                else:
                    # if epoch == 0:
                    #     for data in val_dataset:
                    #         if name_mod != 'PointNet':
                    #             data.edge_index = nng.radius_graph(x = data.pos.to(device), r = hparams['r'], loop = True, max_num_neighbors = int(hparams['max_neighbors'])).cpu()

                    #             if name_mod == 'GNO' or name_mod == 'MGNO':
                    #                 x, edge_index = data.x, data.edge_index
                    #                 x_i, x_j = x[edge_index[0], 0:2], x[edge_index[1], 0:2]
                    #                 v_i, v_j = x[edge_index[0], 2:4], x[edge_index[1], 2:4]
                    #                 p_i, p_j = x[edge_index[0], 4:5], x[edge_index[1], 4:5]
                    #                 v_inf = torch.linalg.norm(v_i, dim = 1, keepdim = True)
                    #                 sdf_i, sdf_j = x[edge_index[0], 5:6], x[edge_index[1], 5:6]
                    #                 normal_i, normal_j = x[edge_index[0], 6:8], x[edge_index[1], 6:8]

                    #                 data.edge_attr = torch.cat([x_i - x_j, v_i - v_j, p_i - p_j, sdf_i, sdf_j, v_inf, normal_i, normal_j], dim = 1)
                    #     val_loader = DataLoader(val_dataset, batch_size = 1, shuffle = False)
                    _, _, val_surf_var, val_vol_var, val_surf, val_vol = test(device, model, val_loader, criterion)
                val_loss = val_surf + val_vol
                val_surf_list.append(val_surf)
                val_vol_list.append(val_vol)
                val_surf_var_list.append(val_surf_var)
                val_vol_var_list.append(val_vol_var)
                pbar_train.set_postfix(train_loss = train_loss, loss_surf = loss_surf, val_loss = val_loss, val_surf = val_surf)
            else:
                pbar_train.set_postfix(train_loss = train_loss, loss_surf = loss_surf, val_loss = val_loss, val_surf = val_surf)
        else:
            pbar_train.set_postfix(train_loss = train_loss, loss_surf = loss_surf)

    loss_surf_var_list = np.array(loss_surf_var_list)
    loss_vol_var_list = np.array(loss_vol_var_list)
    val_surf_var_list = np.array(val_surf_var_list)
    val_vol_var_list = np.array(val_vol_var_list)

    end = time.time()
    time_elapsed = end - start
    params_model = get_nb_trainable_params(model).astype('float')
    print('Number of parameters:', params_model)
    print('Time elapsed: {0:.2f} seconds'.format(time_elapsed))
    torch.save(model, path + 'model')

    sns.set()
    fig_train_surf, ax_train_surf = plt.subplots(figsize = (20, 5))
    ax_train_surf.plot(train_loss_surf_list, label = 'Mean loss')
    ax_train_surf.plot(loss_surf_var_list[:, 0], label = r'$v_x$ loss'); ax_train_surf.plot(loss_surf_var_list[:, 1], label = r'$v_y$ loss')
    ax_train_surf.plot(loss_surf_var_list[:, 2], label = r'$p$ loss'); ax_train_surf.plot(loss_surf_var_list[:, 3], label = r'$\nu_t$ loss')
    ax_train_surf.set_xlabel('epochs')
    ax_train_surf.set_yscale('log')
    ax_train_surf.set_title('Train losses over the surface')
    ax_train_surf.legend(loc = 'best')
    fig_train_surf.savefig(path + 'train_loss_surf.png', dpi = 150, bbox_inches = 'tight')

    fig_train_vol, ax_train_vol = plt.subplots(figsize = (20, 5))
    ax_train_vol.plot(train_loss_vol_list, label = 'Mean loss')
    ax_train_vol.plot(loss_vol_var_list[:, 0], label = r'$v_x$ loss'); ax_train_vol.plot(loss_vol_var_list[:, 1], label = r'$v_y$ loss')
    ax_train_vol.plot(loss_vol_var_list[:, 2], label = r'$p$ loss'); ax_train_vol.plot(loss_vol_var_list[:, 3], label = r'$\nu_t$ loss')
    ax_train_vol.set_xlabel('epochs')
    ax_train_vol.set_yscale('log')
    ax_train_vol.set_title('Train losses over the volume')
    ax_train_vol.legend(loc = 'best')
    fig_train_vol.savefig(path + 'train_loss_vol.png', dpi = 150, bbox_inches = 'tight')

    if val_iter is not None:
        fig_val_surf, ax_val_surf = plt.subplots(figsize = (20, 5))
        ax_val_surf.plot(val_surf_list, label = 'Mean loss')
        ax_val_surf.plot(val_surf_var_list[:, 0], label = r'$v_x$ loss'); ax_val_surf.plot(val_surf_var_list[:, 1], label = r'$v_y$ loss')
        ax_val_surf.plot(val_surf_var_list[:, 2], label = r'$p$ loss'); ax_val_surf.plot(val_surf_var_list[:, 3], label = r'$\nu_t$ loss')
        ax_val_surf.set_xlabel('epochs')
        ax_val_surf.set_yscale('log')
        ax_val_surf.set_title('Validation losses over the surface')
        ax_val_surf.legend(loc = 'best')
        fig_val_surf.savefig(path + 'val_loss_surf.png', dpi = 150, bbox_inches = 'tight')

        fig_val_vol, ax_val_vol = plt.subplots(figsize = (20, 5))
        ax_val_vol.plot(val_vol_list, label = 'Mean loss')
        ax_val_vol.plot(val_vol_var_list[:, 0], label = r'$v_x$ loss'); ax_val_vol.plot(val_vol_var_list[:, 1], label = r'$v_y$ loss')
        ax_val_vol.plot(val_vol_var_list[:, 2], label = r'$p$ loss'); ax_val_vol.plot(val_vol_var_list[:, 3], label = r'$\nu_t$ loss')
        ax_val_vol.set_xlabel('epochs')
        ax_val_vol.set_yscale('log')
        ax_val_vol.set_title('Validation losses over the volume')
        ax_val_vol.legend(loc = 'best')
        fig_val_vol.savefig(path + 'val_loss_vol.png', dpi = 150, bbox_inches = 'tight');
        
        if val_iter is not None:
            with open(path + 'log.json', 'a') as f:
                json.dump(
                    {
                        'regression': 'Total',
                        'loss': 'MSE',
                        'nb_parameters': params_model,
                        'time_elapsed': time_elapsed,
                        'hparams': hparams,
                        'train_loss_surf': train_loss_surf_list[-1],
                        'train_loss_surf_var': loss_surf_var_list[-1],
                        'train_loss_vol': train_loss_vol_list[-1],
                        'train_loss_vol_var': loss_vol_var_list[-1],
                        'val_loss_surf': val_surf_list[-1],
                        'val_loss_surf_var': val_surf_var_list[-1],
                        'val_loss_vol': val_vol_list[-1],
                        'val_loss_vol_var': val_vol_var_list[-1],
                    }, f, indent = 12, cls = NumpyEncoder
                )

    return model

In [3]:
import numpy as np
import pyvista as pv
from reorganize import reorganize
from math import *
import torch
from torch_geometric.data import Data
from matplotlib import pyplot as plt
from scipy.spatial.distance import cdist
from tqdm import tqdm

def cell_sampling_2d(cell_points, cell_attr = None):
    '''
    Sample points in a two dimensional cell via parallelogram sampling and triangle interpolation via barycentric coordinates. The vertices have to be ordered in a certain way.

    Args:
        cell_points (array): Vertices of the 2 dimensional cells. Shape (N, 4) for N cells with 4 vertices.
        cell_attr (array, optional): Features of the vertices of the 2 dimensional cells. Shape (N, 4, k) for N cells with 4 edges and k features. 
            If given shape (N, 4) it will resize it automatically in a (N, 4, 1) array. Default: ``None``
    '''
    # Sampling via triangulation of the cell and parallelogram sampling
    v0, v1 = cell_points[:, 1] - cell_points[:, 0], cell_points[:, 3] - cell_points[:, 0]
    v2, v3 = cell_points[:, 3] - cell_points[:, 2], cell_points[:, 1] - cell_points[:, 2]  
    a0, a1 = np.abs(np.linalg.det(np.hstack([v0[:, :2], v1[:, :2]]).reshape(-1, 2, 2))), np.abs(np.linalg.det(np.hstack([v2[:, :2], v3[:, :2]]).reshape(-1, 2, 2)))
    p = a0/(a0 + a1)
    index_triangle = np.random.binomial(1, p)[:, None]
    u = np.random.uniform(size = (len(p), 2))
    sampled_point = index_triangle*(u[:, 0:1]*v0 + u[:, 1:2]*v1) + (1 - index_triangle)*(u[:, 0:1]*v2 + u[:, 1:2]*v3)
    sampled_point_mirror = index_triangle*((1 - u[:, 0:1])*v0 + (1 - u[:, 1:2])*v1) + (1 - index_triangle)*((1 - u[:, 0:1])*v2 + (1 - u[:, 1:2])*v3)
    reflex = (u.sum(axis = 1) > 1)
    sampled_point[reflex] = sampled_point_mirror[reflex]

    # Interpolation on a triangle via barycentric coordinates
    if cell_attr is not None:
        t0, t1, t2 = np.zeros_like(v0), index_triangle*v0 + (1 - index_triangle)*v2, index_triangle*v1 + (1 - index_triangle)*v3
        w = (t1[:, 1] - t2[:, 1])*(t0[:, 0] - t2[:, 0]) + (t2[:, 0] - t1[:, 0])*(t0[:, 1] - t2[:, 1])
        w0 = (t1[:, 1] - t2[:, 1])*(sampled_point[:, 0] - t2[:, 0]) + (t2[:, 0] - t1[:, 0])*(sampled_point[:, 1] - t2[:, 1])
        w1 = (t2[:, 1] - t0[:, 1])*(sampled_point[:, 0] - t2[:, 0]) + (t0[:, 0] - t2[:, 0])*(sampled_point[:, 1] - t2[:, 1])
        w0, w1 = w0/w, w1/w
        w2 = 1 - w0 - w1
        
        if len(cell_attr.shape) == 2:
            cell_attr = cell_attr[:, :, None]
        attr0 = index_triangle*cell_attr[:, 0] + (1 - index_triangle)*cell_attr[:, 2]
        attr1 = index_triangle*cell_attr[:, 1] + (1 - index_triangle)*cell_attr[:, 1]
        attr2 = index_triangle*cell_attr[:, 3] + (1 - index_triangle)*cell_attr[:, 3]
        sampled_attr = w0[:, None]*attr0 + w1[:, None]*attr1 + w2[:, None]*attr2

    sampled_point += index_triangle*cell_points[:, 0] + (1 - index_triangle)*cell_points[:, 2]    

    return np.hstack([sampled_point[:, :2], sampled_attr]) if cell_attr is not None else sampled_point[:, :2]

def cell_sampling_1d(line_points, line_attr = None):
    '''
    Sample points in a one dimensional cell via linear sampling and interpolation.

    Args:
        line_points (array): Edges of the 1 dimensional cells. Shape (N, 2) for N cells with 2 edges.
        line_attr (array, optional): Features of the edges of the 1 dimensional cells. Shape (N, 2, k) for N cells with 2 edges and k features.
            If given shape (N, 2) it will resize it automatically in a (N, 2, 1) array. Default: ``None``
    '''
    # Linear sampling
    u = np.random.uniform(size = (len(line_points), 1))
    sampled_point = u*line_points[:, 0] + (1 - u)*line_points[:, 1]

    # Linear interpolation
    if line_attr is not None:   
        if len(line_attr.shape) == 2:
            line_attr = line_attr[:, :, None]
        sampled_attr = u*line_attr[:, 0] + (1 - u)*line_attr[:, 1]

    return np.hstack([sampled_point[:, :2], sampled_attr]) if line_attr is not None else sampled_point[:, :2]

def computeSAF(pos,surf_bool,aerofoil):
    # def angle_trunc(a):
    #     while (a < 0.0).any():
    #         a[a<0.0] += (pi * 2)
    #     return a
    def getAngleBetweenPoints(xy_orig, xy_landmark):
        """ Returns angle in radians """
        x_landmark,y_landmark = xy_landmark[:,0],xy_landmark[:,1]
        x_orig,y_orig = xy_orig[:,0],xy_orig[:,1]
        deltaY = y_landmark - y_orig
        deltaX = x_landmark - x_orig
        # return angle_trunc(np.arctan2(deltaY, deltaX)) # Do not use this
        return np.arctan2(deltaY,deltaX)

    def get_closest_points(internalcells, boundarycells):
        dist = cdist(boundarycells,internalcells)
        closests = np.argmin(dist,axis=0)
        return closests, boundarycells[closests]
        
    # closest_point= aerofoil.find_closest_cell(pos)
    closest_point,closest_points_xyz = get_closest_points(pos,pos[surf_bool])
    # print(closest_point)
    # print('shape: ',closest_point.shape,' aerofoil points shape: ',aerofoil.points.shape)
    # closest_points_xyz = aerofoil.points[closest_point]
    # print('closest_points_xyz: ',closest_points_xyz.shape)
    saf_tmp = getAngleBetweenPoints(closest_points_xyz[:,:2],pos[:,:2]) # Angle in radian
    saf_tmp = (saf_tmp)/1.8128
    return saf_tmp

def GcomputeSAF(pos,surf_bool): # Assume pos, surf_bool are torch tensors
    # def angle_trunc(a):
    #     while (a < 0.0).any():
    #         a[a<0.0] += (pi * 2)
    #     return a
    def getAngleBetweenPoints(xy_orig, xy_landmark):
        """ Returns angle in radians """
        x_landmark,y_landmark = xy_landmark[:,0],xy_landmark[:,1]
        x_orig,y_orig = xy_orig[:,0],xy_orig[:,1]
        deltaY = y_landmark - y_orig
        deltaX = x_landmark - x_orig
        return torch.atan2(deltaY,deltaX)

    def get_closest_points(internalcells, boundarycells):
        dist = torch.cdist(boundarycells,internalcells,p=2)
        # print('dist size: ',dist.size())
        closests = torch.argmin(dist,dim=0)
        return closests, boundarycells[closests]
        
    # closest_point= aerofoil.find_closest_cell(pos)
    closest_point,closest_points_xyz = get_closest_points(pos,pos[surf_bool])
    # print(closest_point)
    # print('shape: ',closest_point.shape,' aerofoil points shape: ',aerofoil.points.shape)
    # closest_points_xyz = aerofoil.points[closest_point]
    # print('closest_points_xyz: ',closest_points_xyz.shape)
    saf_tmp = getAngleBetweenPoints(closest_points_xyz[:,:2],pos[:,:2]) # Angle in radian
    # saf_tmp = (saf_tmp)/1.8128 # normalization => subtract mean([-pi,pi]), divide by std([-pi,pi])
    return saf_tmp

def getDSDF(pos, bd, theta_rot, theta_seg, inf=5):
    '''get dSDF representation of geometry given rotation angle, segment angle.'''
    # print('----- DSDF ---------')
    # print('type(pos) & shape(pos): ', type(pos), pos.size())
    # print('type(bd) & shape(bd): ',type(bd), bd.size())
    def order_clockwise(bd_xy):
        c = torch.mean(bd_xy,dim=0)
        h = bd_xy - c
        theta = torch.atan2(h[:,1],h[:,0]) #size MxN
        return torch.flipud(bd_xy[theta.sort()[1]])
    def sameSide(j,i):
        '''Check if boundary points j (Mx2) and all points i (Nx2)
        fall on the same side of the geometry (bool NxM)'''
        # c = torch.tensor([[0.5,0]]).to(j.device) #airfoil internal point
        c = torch.mean(j,dim=0).unsqueeze(0)
        j = order_clockwise(j); j1 = torch.cat([j[1:,:],j[0:1,:]])
        indi, indj = torch.meshgrid(torch.arange(i.size(0)), \
                                torch.arange(j.size(0)), indexing='ij')
        p0 = i[indi]; p1 = j[indj]; p2 = j1[indj]
        dir_i = (p1[:,:,1]-p0[:,:,1])*(p2[:,:,0]-p1[:,:,0])
        dir_i -= (p2[:,:,1]-p1[:,:,1])*(p1[:,:,0]-p0[:,:,0])
        dir_c = (j[:,1]-c[:,1])*(j1[:,0]-j[:,0])
        dir_c -= (j1[:,1]-j[:,1])*(j[:,0]-c[:,0])
        side = ((dir_i>0)==(dir_c<=0)).t() #bool size NxM
        return j,side

    def PDF(mean,x):
        '''chosen PDF to used to weigh points'''
        return torch.ones(x.size()).to(d.device) #if just uniform distribution
        # grad = 1/theta_seg**2;
        # w0 = (x-(mean-theta_seg/2))/(theta_seg**2)
        # w1 = (-x+(mean+theta_seg/2))/(theta_seg**2) #linear
        # return w0*(x<=mean)+w1*(x>mean) #if linear distribution
    
    def intPDF(mean,mRange):
        '''integral of chosen PDF used to weigh points'''
        return (mRange[1]-mRange[0]) #for uniform distribution
        # dw = PDF(mean,mRange); grad = 1/theta_seg**2;
        # w0 = 0.5*dw*(mRange-(mean-theta_seg/2))
        # w1 = 1 - (0.5*dw*((mean+theta_seg/2)-mRange)) #linear
        # w = (w0*(mRange<=mean)+w1*(mRange>mean))*(dw>=0)
        # return w[1]-w[0] #if linear distribution

    dSDF = []; j = pos[bd]; j, side = sameSide(j,pos)
    #Compute dSDF for ever segment centre theta_cen
    for theta_cen in torch.arange(0,2*torch.pi,theta_rot):
        #Compute segment range theta_ran
        theta_ran = torch.tensor([theta_cen-(theta_seg/2), theta_cen+(theta_seg/2)])
        dSDF_tCen = []
        
        #Compute dSDF_i for every node i in V
        h = j[:,0:1] - pos.t()[0:1,:] #size MxN
        k = j[:,1:] - pos.t()[1:,:] #size MxN
        theta_ij = torch.atan2(k,h) #size MxN
        theta_ij = theta_ij%(2*torch.pi) if theta_ran[0]>=0 else theta_ij
        
        #Compute distance from point i to every bd point within segment range AND same side
        ind = (theta_ran[0]<=theta_ij)*(theta_ij<=theta_ran[1])*side
        d = torch.sqrt(k*k+h*h); d[d>inf]=inf #size MxN
        w = PDF(theta_cen,theta_ij) #size MxN
        dSDF_i = torch.sum((d*w)*ind,dim=0) #dSDF_i = torch.sum((d*ind),dim=0)
        w = torch.sum((w*ind),dim=0) #w=torch.sum(ind,dim=0) #sum of discrete weights
        
        #minimum angle range from point i to point j
        theta_minR = torch.zeros(2,theta_ij.size(1)).to(dSDF_i.device)
        ind = (theta_ran[0]<=theta_ij)*(theta_ij<=theta_ran[1]) # in segment range (but any side)
        theta_ij[~ind] = 2*torch.pi; theta_minR[0] = torch.min(theta_ij,dim=0)[0]
        theta_ij[~ind] = -2*torch.pi; theta_minR[1] = torch.max(theta_ij,dim=0)[0]
        
        #compute weight of minimum angle segment w_tMin
        w_tMin = intPDF(theta_cen,theta_minR)/intPDF(theta_cen,theta_ran)
        w_tMin[w_tMin<0] = 0
        dSDF_i = torch.nan_to_num(dSDF_i/w,nan=0)
        dSDF_tCen = w_tMin*dSDF_i + (1-w_tMin)*inf
        dSDF.append(dSDF_tCen.clone())
    dSDF = torch.stack(dSDF)
    return dSDF
  

def Dataset(set, norm = False, coef_norm = None, crop = None, sample = None, n_boot = int(5e5), surf_ratio = .1, \
    use_saf=False,use_dsdf=False, manifest_dsdf=None,manifest_saf=None):
    '''
    Create a list of simulation to input in a PyTorch Geometric DataLoader. Simulation are transformed by keeping vertices of the CFD mesh or 
    by sampling (uniformly or via the mesh density) points in the simulation cells.

    Args:
        set (list): List of geometry names to include in the dataset.
        norm (bool, optional): If norm is set to ``True``, the mean and the standard deviation of the dataset will be computed and returned. 
            Moreover, the dataset will be normalized by these quantities. Ignored when ``coef_norm`` is not None. Default: ``False``
        coef_norm (tuple, optional): This has to be a tuple of the form (mean input, std input, mean output, std ouput) if not None. 
            The dataset generated will be normalized by those quantites. Default: ``None``
        crop (list, optional): List of the vertices of the rectangular [xmin, xmax, ymin, ymax] box to crop simulations. Default: ``None``
        sample (string, optional): Type of sampling. If ``None``, no sampling strategy is applied and the nodes of the CFD mesh are returned.
            If ``uniform`` or ``mesh`` is chosen, uniform or mesh density sampling is applied on the domain. Default: ``None``
        n_boot (int, optional): Used only if sample is not None, gives the size of the sampling for each simulation. Defaul: ``int(5e5)``
        surf_ratio (float, optional): Used only if sample is not None, gives the ratio of point over the airfoil to sample with respect to point
            in the volume. Default: ``0.1``
    '''
    if norm and coef_norm is not None:
        raise ValueError('If coef_norm is not None and norm is True, the normalization will be done via coef_norm')

    dataset = []

    for k, s in enumerate(tqdm(set)):
        # Get the 3D mesh, add the signed distance function and slice it to return in 2D
        internal = pv.read('Dataset/' + s + '/' + s + '_internal.vtu')
        aerofoil = pv.read('Dataset/' + s + '/' + s + '_aerofoil.vtp')
        internal = internal.compute_cell_sizes(length = False, volume = False)
        # Cropping if needed, crinkle is True.
        if crop is not None:
            bounds = (crop[0], crop[1], crop[2], crop[3], 0, 1)
            internal = internal.clip_box(bounds = bounds, invert = False, crinkle = True)

        # If sampling strategy is chosen, it will sample points in the cells of the simulation instead of directly taking the nodes of the mesh.
        if sample is not None:
            # Sample on a new point cloud
            if sample == 'uniform': # Uniform sampling strategy
                p = internal.cell_data['Area']/internal.cell_data['Area'].sum()
                sampled_cell_indices = np.random.choice(internal.n_cells, size = n_boot, p = p)
                surf_p = aerofoil.cell_data['Length']/aerofoil.cell_data['Length'].sum()
                sampled_line_indices = np.random.choice(aerofoil.n_cells, size = int(n_boot*surf_ratio), p = surf_p)
            elif sample == 'mesh': # Sample via mesh density
                sampled_cell_indices = np.random.choice(internal.n_cells, size = n_boot)
                sampled_line_indices = np.random.choice(aerofoil.n_cells, size = int(n_boot*surf_ratio))

            cell_dict = internal.cells.reshape(-1, 5)[sampled_cell_indices, 1:]
            cell_points = internal.points[cell_dict]            
            line_dict = aerofoil.lines.reshape(-1, 3)[sampled_line_indices, 1:]
            line_points = aerofoil.points[line_dict]

            # Geometry information
            geom = -internal.point_data['implicit_distance'][cell_dict, None] # Signed distance function
            Uinf, alpha = float(s.split('_')[2]), float(s.split('_')[3])*np.pi/180
            # u = (np.array([np.cos(alpha), np.sin(alpha)])*Uinf).reshape(1, 2)*(internal.point_data['U'][cell_dict, :1] != 0)
            u = (np.array([np.cos(alpha), np.sin(alpha)])*Uinf).reshape(1, 2)*np.ones_like(internal.point_data['U'][cell_dict, :1])
            normal = np.zeros_like(u)

            surf_geom = np.zeros_like(aerofoil.point_data['U'][line_dict, :1])
            # surf_u = np.zeros_like(aerofoil.point_data['U'][line_dict, :2])
            surf_u = (np.array([np.cos(alpha), np.sin(alpha)])*Uinf).reshape(1, 2)*np.ones_like(aerofoil.point_data['U'][line_dict, :1])
            surf_normal = -aerofoil.point_data['Normals'][line_dict, :2]

            attr = np.concatenate([u, geom, normal, internal.point_data['U'][cell_dict, :2], 
                internal.point_data['p'][cell_dict, None], internal.point_data['nut'][cell_dict, None]], axis = -1)
            surf_attr = np.concatenate([surf_u, surf_geom, surf_normal, aerofoil.point_data['U'][line_dict, :2], 
                aerofoil.point_data['p'][line_dict, None], aerofoil.point_data['nut'][line_dict, None]], axis = -1)
            sampled_points = cell_sampling_2d(cell_points, attr)
            surf_sampled_points = cell_sampling_1d(line_points, surf_attr)

            # Define the inputs and the targets
            pos = sampled_points[:, :2]
            init = sampled_points[:, :7]
            target = sampled_points[:, 7:]
            surf_pos = surf_sampled_points[:, :2]
            surf_init = surf_sampled_points[:, :7]
            surf_target = surf_sampled_points[:, 7:]

            # if cell_centers:
            #     centers = internal.ptc().cell_centers()
            #     surf_centers = aerofoil.cell_centers()

            #     geom = -centers.cell_data['implicit_distance'][:, None] # Signed distance function
            #     Uinf, alpha = float(s.split('_')[2]), float(s.split('_')[3])*np.pi/180
            #     u = (np.array([np.cos(alpha), np.sin(alpha)])*Uinf).reshape(1, 2)*np.ones_like(internal.cell_data['U'][:, :1])
            #     normal = np.zeros_like(u)

            #     surf_geom = np.zeros_like(surf_centers.cell_data['U'][:, :1])
            #     # surf_u = np.zeros_like(surf_centers.cell_data['U'][:, :2])
            #     surf_u = (np.array([np.cos(alpha), np.sin(alpha)])*Uinf).reshape(1, 2)*np.ones_like(surf_centers.cell_data['U'][:, :1])
            #     surf_normal = -aerofoil.cell_data['Normals'][:, :2]

            #     attr = np.concatenate([u, geom, normal,
            #         internal.cell_data['U'][:, :2], internal.cell_data['p'][:, None], internal.cell_data['nut'][:, None]], axis = -1)
            #     surf_attr = np.concatenate([surf_u, surf_geom, surf_normal,
            #         aerofoil.cell_data['U'][:, :2], aerofoil.cell_data['p'][:, None], aerofoil.cell_data['nut'][:, None]], axis = -1)

            #     bool_centers = np.concatenate([np.ones_like(centers.points[:, 0]), np.zeros_like(pos[:, 0])], axis = 0)
            #     surf_bool_centers = np.concatenate([np.ones_like(surf_centers.points[:, 0]), np.zeros_like(surf_pos[:, 0])], axis = 0)
            #     pos = np.concatenate([centers.points[:, :2], pos], axis = 0)
            #     init = np.concatenate([np.concatenate([centers.points[:, :2], attr[:, :6]], axis = 1), init], axis = 0)
            #     target = np.concatenate([attr[:, 6:], target], axis = 0)
            #     surf_pos = np.concatenate([surf_centers.points[:, :2], surf_pos], axis = 0)
            #     surf_init = np.concatenate([np.concatenate([surf_centers.points[:, :2], surf_attr[:, :6]], axis = 1), surf_init], axis = 0)
            #     surf_target = np.concatenate([surf_attr[:, 6:], surf_target], axis = 0)

            #     centers = torch.cat([torch.tensor(bool_centers), torch.tensor(surf_bool_centers)], dim = 0)

            # Put everything in tensor
            surf = torch.cat([torch.zeros(len(pos)), torch.ones(len(surf_pos))], dim = 0)
            pos = torch.cat([torch.tensor(pos, dtype = torch.float), torch.tensor(surf_pos, dtype = torch.float)], dim = 0) 
            x = torch.cat([torch.tensor(init, dtype = torch.float), torch.tensor(surf_init, dtype = torch.float)], dim = 0)
            y = torch.cat([torch.tensor(target, dtype = torch.float), torch.tensor(surf_target, dtype = torch.float)], dim = 0)            

        else: # Keep the mesh nodes
            surf_bool = (internal.point_data['U'][:, 0] == 0)

            # print('surf_bool: ',surf_bool)
            geom = -internal.point_data['implicit_distance'][:, None] # Signed distance function
            # print('sdf: ',geom)
            Uinf, alpha = float(s.split('_')[2]), float(s.split('_')[3])*np.pi/180
            # print('Uinf: ',Uinf)
            # print('alpha: ',alpha)
            # u = (np.array([np.cos(alpha), np.sin(alpha)])*Uinf).reshape(1, 2)*(internal.point_data['U'][:, :1] != 0)
            u = (np.array([np.cos(alpha), np.sin(alpha)])*Uinf).reshape(1, 2)*np.ones_like(internal.point_data['U'][:, :1])
            # print('u: ',u)
            normal = np.zeros_like(u)
            normal[surf_bool] = reorganize(aerofoil.points[:, :2], internal.points[surf_bool, :2], -aerofoil.point_data['Normals'][:, :2])
            # print('internal.point_data keys: ',internal.point_data.keys())

            # print('u , geom, normal, internal.point_data[u] [p] [nut] shapes:')
            # print(u.shape)
            # print(geom.shape)
            # print(normal.shape)
            # print(internal.point_data['U'].shape)
            # print(internal.point_data['p'].shape)
            # print(internal.point_data['nut'].shape)
            attr = np.concatenate([u, geom, normal,
                internal.point_data['U'][:, :2], internal.point_data['p'][:, None], internal.point_data['nut'][:, None]], axis = -1)

            # print('attr shape: ',attr.shape)
            # print('first 5 columns are x')
            # print(attr.shape[1]-5,' columns are y')
            pos = internal.points[:, :2] # cx,cy
            init = np.concatenate([pos, attr[:, :5]], axis = 1) # cx,cy, ux,uy,sdf,nx,ny
            target = attr[:, 5:] # U
            # saf = computeSAF(internal.points[:,:3],surf_bool,aerofoil)
            # Plotting SDF
            # if k==0:
            #     F = ( (pos[:,0]<2) & (pos[:,0]>-1)  & (pos[:,1]>-0.5) & (pos[:,1]<0.5))
            #     xyF = pos[F]
            #     sdfF = geom[F]
            #     plt.scatter(xyF[:,0],xyF[:,1],c=sdfF,s=1,cmap = 'viridis'); plt.colorbar(); plt.savefig(s+'_SDF.png'); plt.clf()
            # # Plotting SAF
            # if k<4:
            #     F = ( (pos[:,0]<2) & (pos[:,0]>-1)  & (pos[:,1]>-0.5) & (pos[:,1]<0.5))
            #     xyF = pos[F]
            #     safF = saf[F]
            #     plt.scatter(xyF[:,0],xyF[:,1],c=safF,s=1,cmap = 'viridis'); plt.colorbar(); plt.savefig(s+'_SAF.png'); plt.clf()
            # Put everything in tensor
            surf = torch.tensor(surf_bool)
            pos = torch.tensor(pos, dtype = torch.float)
            x = torch.tensor(init, dtype = torch.float)
            y = torch.tensor(target, dtype = torch.float) # ux,uy,sdf,nx,ny
            #print('target y :',y.shape)

        if norm and coef_norm is None:
            if k == 0:
                old_length = init.shape[0]
                mean_in = init.mean(axis = 0, dtype = np.double)
                mean_out = target.mean(axis = 0, dtype = np.double)
            else:
                new_length = old_length + init.shape[0]
                mean_in += (init.sum(axis = 0, dtype = np.double) - init.shape[0]*mean_in)/new_length
                mean_out += (target.sum(axis = 0, dtype = np.double) - init.shape[0]*mean_out)/new_length
                old_length = new_length 

        # Graph definition
        # if cell_centers:
        #     data = Data(pos = pos, x = x, y = y, surf = surf.bool(), centers = centers.bool())
        # else:
        #     data = Data(pos = pos, x = x, y = y, surf = surf.bool())
        if use_saf:
            saf = torch.load(manifest_saf[s])
        else:
            saf = None 
        if (use_dsdf):
            dsdf = torch.load(manifest_dsdf[s])
        else:
            dsdf = None
        data = Data(pos = pos, x = x, y = y, surf = surf.bool(),saf = saf,dsdf=dsdf)
        dataset.append(data)

    if norm and coef_norm is None:
        # Compute normalization
        mean_in = mean_in.astype(np.single)
        mean_out = mean_out.astype(np.single)
        # Umean = np.linalg.norm(data.x[:, 2:4], axis = 1).mean()     
        for k, data in enumerate(dataset):
            # data.x = data.x/torch.tensor([6, 6, Umean, Umean, 6, 1, 1], dtype = torch.float)
            # data.y = data.y/torch.tensor([Umean, Umean, .5*Umean**2, Umean], dtype = torch.float)

            if k == 0:
                old_length = data.x.numpy().shape[0]
                std_in = ((data.x.numpy() - mean_in)**2).sum(axis = 0, dtype = np.double)/old_length
                std_out = ((data.y.numpy() - mean_out)**2).sum(axis = 0, dtype = np.double)/old_length
            else:
                new_length = old_length + data.x.numpy().shape[0]
                std_in += (((data.x.numpy() - mean_in)**2).sum(axis = 0, dtype = np.double) - data.x.numpy().shape[0]*std_in)/new_length
                std_out += (((data.y.numpy() - mean_out)**2).sum(axis = 0, dtype = np.double) - data.x.numpy().shape[0]*std_out)/new_length
                old_length = new_length
        
        std_in = np.sqrt(std_in).astype(np.single)
        std_out = np.sqrt(std_out).astype(np.single)

        # Normalize
        for data in dataset:
            data.x = (data.x - mean_in)/(std_in + 1e-8)
            data.y = (data.y - mean_out)/(std_out + 1e-8)

        coef_norm = (mean_in, std_in, mean_out, std_out)     
        dataset = (dataset, coef_norm)   
    
    elif coef_norm is not None:
        # Normalize
        for data in dataset:
            # data.x = data.x/torch.tensor([6, 6, coef_norm[-1], coef_norm[-1], 6, 1, 1], dtype = torch.float)
            # data.y = data.y/torch.tensor([coef_norm[-1], coef_norm[-1], .5*coef_norm[-1]**2, coef_norm[-1]], dtype = torch.float)
            data.x = (data.x - coef_norm[0])/(coef_norm[1] + 1e-8)
            data.y = (data.y - coef_norm[2])/(coef_norm[3] + 1e-8)
    
    return dataset


In [13]:
GraphSAGE:
  encoder: [7, 64, 64, 8]
  decoder: [8, 64, 64, 4]

  nb_hidden_layers: 3
  size_hidden_layers: 64
  batch_size: 1
  nb_epochs: 398
  lr: 0.001
  max_neighbors: 64
  bn_bool: True
  subsampling: 32000
  r: 0.05

PointNet:
  encoder: [7, 64, 64, 8]
  decoder: [8, 64, 64, 4]

  base_nb: 8
  batch_size: 1
  nb_epochs: 398
  lr: 0.001
  subsampling: 32000

MLP:
  encoder: [7, 64, 64, 8]
  decoder: [8, 64, 64, 4]

  nb_hidden_layers: 3
  size_hidden_layers: 64
  batch_size: 1
  nb_epochs: 398
  lr: 0.001
  bn_bool: True
  subsampling: 32000

GUNet:
  #encoder: [7, 64, 64, 8] #EDITED
  encoder: [16, 64, 64, 8] #EDITED####
  decoder: [8, 64, 64, 4]

  layer: 'SAGE'
  pool: 'random'
  nb_scale: 5
  pool_ratio: [.5, .5, .5, .5]
  list_r: [.05, .2, .5, 1, 10]
  size_hidden_layers: 8
  batchnorm: True
  res: False

  batch_size: 1
  nb_epochs: 398
  lr: 0.001
  max_neighbors: 64
  subsampling: 32000
  r: 0.05

GUNetFVGCN:
  #encoder: [7, 64, 64, 8] #EDITED
  encoder: [16, 64, 64, 8] #EDITED####
  decoder: [8, 64, 64, 4]

  layer: 'SAGE'
  pool: 'random'
  nb_scale: 5
  pool_ratio: [.5, .5, .5, .5]
  list_r: [.05, .2, .5, 1, 10]
  size_hidden_layers: 8
  batchnorm: True
  res: False

  batch_size: 1
  nb_epochs: 398
  lr: 0.001
  max_neighbors: 64
  subsampling: 32000
  r: 0.05

SyntaxError: invalid syntax (2149973580.py, line 1)

In [4]:
#@title Main script, EDITED to use dSDF data and train GUNet with aSGCN_conv

#edited from: https://github.com/Extrality/AirfRANS/blob/main/main.py

import argparse, yaml, os, json, glob
import torch
import train, metrics
from dataset import Dataset

import numpy as np

parser = argparse.ArgumentParser()
parser.add_argument('model', help = 'The model you want to train, choose between MLP, GraphSAGE, PointNet, GUNet', type = str)
parser.add_argument('-n', '--nmodel', help = 'Number of trained models for standard deviation estimation (default: 1)', default = 1, type = int)
parser.add_argument('-w', '--weight', help = 'Weight in front of the surface loss (default: 1)', default = 1, type = float)
parser.add_argument('-t', '--task', help = 'Task to train on. Choose between "full", "scarce", "reynolds" and "aoa" (default: full)', default = 'full', type = str)
parser.add_argument('-s', '--score', help = 'If you want to compute the score of the models on the associated test set. (default: 0)', default = 0, type = int)
parser.add_argument('-saf','--saf',default = 1, type = int, help = 'Use saf for training')
parser.add_argument('-dsdf','--dsdf', default = 1, type=int, help = 'Use dsdf for training')
args = parser.parse_args()

with open('/home/jessica/Downloads/submission/AirfRANS/Dataset/manifest.json', 'r') as f:
    manifest = json.load(f)
with open('/home/jessica/Downloads/submission/AirfRANS/Dataset/manifest_dsdf.json', 'r') as f:
    manifest_dsdf = json.load(f)
with open('/home/jessica/Downloads/submission/AirfRANS/Dataset/manifest_saf.json', 'r') as f:
    manifest_saf = json.load(f)


manifest_train = manifest[args.task + '_train']
test_dataset = manifest[args.task + '_test'] if args.task != 'scarce' else manifest['full_test']
n = int(.1*len(manifest_train))
train_dataset = manifest_train[:-n]
val_dataset = manifest_train[-n:]

# if os.path.exists('Dataset/train_dataset'):
#     train_dataset = torch.load('Dataset/train_dataset')
#     val_dataset = torch.load('Dataset/val_dataset')
#     coef_norm = torch.load('Dataset/normalization')
# else:
train_dataset, coef_norm = Dataset(train_dataset, norm = True, sample = None, \
        use_saf = args.saf,use_dsdf = args.dsdf, manifest_dsdf = manifest_dsdf,manifest_saf = manifest_saf)
# torch.save(train_dataset, 'Dataset/train_dataset')
# torch.save(coef_norm, 'Dataset/normalization')
val_dataset = Dataset(val_dataset, sample = None, coef_norm = coef_norm)
# torch.save(val_dataset, 'Dataset/val_dataset')

# Cuda
use_cuda = torch.cuda.is_available()
device = 'cuda:1' if use_cuda else 'cpu' #EDITED####
if use_cuda:
    print('Using: ',device) #print('Using GPU')
else:
    print('Using CPU')

#with open('params.yaml', 'r') as f: # hyperparameters of the model
with open('paramsNEW.yaml', 'r') as f: #EDITED####
    hparams = yaml.safe_load(f)[args.model]

from models.MLP import MLP
models = []
for i in range(args.nmodel):
    encoder = MLP(hparams['encoder'], batch_norm = False)
    decoder = MLP(hparams['decoder'], batch_norm = False)

    if args.model == 'GraphSAGE':
        from models.GraphSAGE import GraphSAGE
        model = GraphSAGE(hparams, encoder, decoder)
    
    elif args.model == 'PointNet':
        from models.PointNet import PointNet
        model = PointNet(hparams, encoder, decoder)

    elif args.model == 'MLP':
        from models.NN import NN
        model = NN(hparams, encoder, decoder)

    elif args.model == 'GUNet':
        from models.GUNet import GUNet
        model = GUNet(hparams, encoder, decoder)
        
    elif args.model == 'GUNetFVGCN':
        from models.GUNetFVGCN import GUNetFVGCN
        model = GUNetFVGCN(hparams, encoder, decoder)

    path = 'metrics/' # path where you want to save log and figures
    model = train.main(device, train_dataset, val_dataset, model, hparams, path, 
                criterion = 'MSE_weighted', val_iter = None, reg = args.weight, name_mod = args.model, val_sample = True,\
                      use_saf = args.saf, use_dsdf = args.dsdf)
    models.append(model)
torch.save(models, args.model+'_aSGN_SAF_dSDF') #EDITED####

if bool(args.score):
    s = args.task + '_test' if args.task != 'scarce' else 'full_test'
    coefs = metrics.Results_test(device, [models], hparams, coef_norm, n_test = 3, path_in = 'Dataset/', criterion = 'MSE', s = s)
    # models can be a stack of the same model (for example MLP) on the task s, if you have another stack of another model (for example GraphSAGE)
    # you can put in model argument [models_MLP, models_GraphSAGE] and it will output the results for both models (mean and std) in an ordered array.
    np.save('scores/' + args.task + '/true_coefs', coefs[0])
    np.save('scores/' + args.task + '/pred_coefs_mean', coefs[1])
    np.save('scores/' + args.task + '/pred_coefs_std', coefs[2])
    for n, file in enumerate(coefs[3]):
        np.save('scores/' + args.task + '/true_surf_coefs_' + str(n), file)
    for n, file in enumerate(coefs[4]):
        np.save('scores/' + args.task + '/surf_coefs_' + str(n), file)
    np.save('scores/' + args.task + '/true_bls', coefs[5])
    np.save('scores/' + args.task + '/bls', coefs[6])
    for aero in glob.glob('airFoil2D*'):
        os.rename(aero, 'scores/' + args.task + '/' + aero)
    os.rename('score.json', 'scores/' + args.task + '/score.json')

usage: ipykernel_launcher.py [-h] [-n NMODEL] [-w WEIGHT] [-t TASK] [-s SCORE]
                             [-saf SAF] [-dsdf DSDF]
                             model
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

/home/jessica/anaconda3/envs/airfrans_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
lin_in = torch.nn.Linear(0, 5)
print(lin_in)

Linear(in_features=0, out_features=5, bias=True)


/home/jessica/anaconda3/envs/airfrans_env/lib/python3.9/site-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [1]:
#RUN THIS:

#cd /home/jessica/Downloads/submission/AirfRANS/

"""
# python mainNEW.py GUNet -t scarce -n 1 -s 1 -p half -cuda cuda:2
# python mainNEW.py GUNetSGraphSAGE_SAF_dSDF -t scarce -n 1 -s 1 -p half -cuda cuda:2
# python mainNEW_wip.py GNetFVGraphSAGE_FVnoS_SAF_dSDF -t scarce -n 1 -s 1 -cuda cuda:2

# python mainNEWNS.py GNetFVnewGraphSAGE_FV_SAF_dSDF -t scarce -n 1 -s 1 -p half -cuda cuda:2
# python mainNEWNS.py GNetFVnewGraphSAGE_FV_SAF_dSDF_NS -t scarce -n 1 -s 1 -p half -ns 1 -cuda cuda:2

# python inference.py -t scarce -M GNetFVGraphSAGE_FVnoS_SAF_dSDF -m GNetFVGraphSAGE_FVnoS_SAF_dSDF -saf 1 -dsdf 1
# python inference.py -t scarce -M GUNetSGraphSAGE_SAF_dSDF -m GUNetSGraphSAGE_SAF_dSDF -saf 1 -dsdf 1
# python inference.py -t scarce -M GNetFVnewGraphSAGE_FV_SAF_dSDF -m GNetFVnewGraphSAGE_FV_SAF_dSDF -saf 1 -dsdf 1
"""

# 38556
# python mainNEW_wip.py GUNetGCN -t scarce -n 1 -s 1 -p half -cuda cuda:0
# 39132
# python mainNEW_wip.py GUNetGCN_SAF_dSDF -t scarce -n 1 -s 1 -p half -cuda cuda:1
# 105201
# python mainNEWNS.py GNetFVnewGCN_FVnew_SAF_dSDF -t scarce -n 1 -s 1 -p half -cuda cuda:2

# 160465
# python mainNEWNS.py GNetFVnewGraphSAGE_FV_SAF_dSDF -t full -n 1 -s 1 -p half -cuda cuda:0

#python inference.py -t full -M GNetFVnewGraphSAGE_FV_SAF_dSDF -m Effectiveness_models/GNetFVnewGraphSAGE_FV_SAF_dSDFhalf_fvnew_0.25data_final


###### REBUT #####
# python mainNEW.py GUNet -t scarce -n 1 -s 1 -p half -cuda cuda:1
# python mainNEWNS.py GNetFVnewGraphSAGE_FV_SAF_dSDF -t scarce -n 1 -s 1 -p half -cuda cuda:2

# python inference.py -t scarce -M GUNet -m GUNethalf_w1_final
# python inference.py -t scarce -M GNetFVnewGraphSAGE_FV_SAF_dSDF -m GNetFVnewGraphSAGE_FV_SAF_dSDF_w1half_final
# python inference.py -t scarce -M GNetFVnewGraphSAGE_FV_SAF_dSDF_w2 -m GNetFVnewGraphSAGE_FV_SAF_dSDF_w2half_final

SyntaxError: invalid syntax (127919933.py, line 25)

In [11]:
import torch
print(torch.__version__)

1.12.0+cu102


In [5]:
with open('/home/jessica/Downloads/submission/AirfRANS/Dataset/manifest.json', 'r') as f:
    manifest = json.load(f)
with open('/home/jessica/Downloads/submission/AirfRANS/Dataset/manifest_dsdf.json', 'r') as f:
    manifest_dsdf = json.load(f)
with open('/home/jessica/Downloads/submission/AirfRANS/Dataset/manifest_saf.json', 'r') as f:
    manifest_saf = json.load(f)


manifest_train = manifest["scarce" + '_train'][:10]
test_dataset = manifest["scarce" + '_test'] if "scarce" != 'scarce' else manifest['full_test']
n = int(.1*len(manifest_train))
train_dataset = manifest_train[:-n]
val_dataset = manifest_train[-n:]

train_dataset, coef_norm = Dataset(train_dataset, norm = True, sample = None, \
        use_saf = 1,use_dsdf = 1, manifest_dsdf = manifest_dsdf,manifest_saf = manifest_saf)
val_dataset = Dataset(val_dataset, sample = None, coef_norm = coef_norm)


100%|███████████████████████████████████████████| 20/20 [00:12<00:00,  1.65it/s]


In [6]:
device = 'cuda:1' if True else 'cpu'

with open('paramsNEW.yaml', 'r') as f: #EDITED####
    hparams = yaml.safe_load(f)['FVGCN_SAF_dSDF']
    
from models.MLP import MLP
if (hparams['encoder'] is not False):
    encoder = MLP(hparams['encoder'], batch_norm = False)
    decoder = MLP(hparams['decoder'], batch_norm = False)
else:
    encoder, decoder = None, None
    
path = 'metrics/'
    
from models.FVGCN import FVGCN
model = FVGCN(hparams, encoder, decoder)


model_parameters = filter(lambda p: p.requires_grad, model.parameters())
print('model_parameters : ',sum([np.prod(p.size()) for p in model_parameters]))

model = train.main(device, train_dataset, val_dataset, model, hparams, path, 
                criterion = 'MSE_weighted', val_iter = None, reg = 1, name_mod = 'FVGCN', val_sample = True,\
                      use_saf = hparams['SAF'], use_dsdf = hparams['dSDF'])

model_parameters :  64232


  0%|      | 1/398 [04:21<28:48:17, 261.20s/it, loss_surf=1.69, train_loss=2.91]


KeyboardInterrupt: 

In [1]:
model = torch.load()

i=0
batch = test_dataset[i]
out = model(batch)

NameError: name 'torch' is not defined